In [1]:
from matching_functions import *

BIBLE_SHINGLE_FOLDER = '/home/spenteco/text_reuse.HOME/bible_pickles/'
EEBO_SHINGLE_FOLDER = '/home/spenteco/0/eebo_shingled/'
METADATA_FILE = 'metadata/EEBO_metadata.tsv'
OUTPUT_FOLDER = 'text_reuse_html_outputs/'

metadata = load_metadata(METADATA_FILE)

In [2]:
all_verses = {
    'KJV': [],
    'Geneva': []
}

all_verses['KJV'] = load_pickle_file(BIBLE_SHINGLE_FOLDER + 'KJV_OTA.pickle')

for p in sorted(glob.glob(BIBLE_SHINGLE_FOLDER + 'geneva/*.pickle')):
    all_verses['Geneva'] += load_pickle_file(p)
    
print(len(all_verses['KJV']))
print(len(all_verses['Geneva']))

31102
31102


In [3]:
print('A97378', metadata['A97378'])**bible_to_verses_from_TCP.ipynb**
print()
print('A10675', metadata['A10675'])
print()

tcp_kjv = load_pickle_file(EEBO_SHINGLE_FOLDER + 'A97378.pickle')
tcp_geneva = load_pickle_file(EEBO_SHINGLE_FOLDER + 'A10675.pickle')

print('Loaded!')

A97378 {'year': '1668', 'author': ' ', 'title': 'The Holy Bible containing the Old Testament and the New / newly translated out of the original tongues and with the former translations diligently compared and revised by His Majesties speciall command ; appointed to be read in churches.|Bible. English. Authorized. 1668.'}

A10675 {'year': '1562', 'author': 'Whittingham, William, d. 1579.|Gilby, Anthony, ca. 1510-1585.|Sampson, Thomas, 1517?-1589.', 'title': 'The Bible and Holy Scriptures conteyned in the Olde and Newe Testament. Translated according to the Ebrue and Greke, and conferred with the best translations in diuers languges. VVith moste profitable annotations vpon all the hard places, and other things of great importance as may appeare in the epistle to the reader|Bible. English. Geneva.'}

Loaded!


In [4]:
from mako.template import Template

def make_matching_bible(my_bible, tcp_bible, output_label):
    
    t = Template(filename='matching_bible_template.html')
    t_counts = Template(filename='matching_bible_template_counts.html')
    
    output_results = []
    exception_results = []
    exception_counts = []
    
    for v in my_bible:

        MAX_GAP_ALLOWED = 5
        MIN_MATCH_LENGTH = int(len(v['non_space_lemmas']) * 0.75)
        
        results = match_two_files(v, 
                                    tcp_bible,
                                    MAX_GAP_ALLOWED, MIN_MATCH_LENGTH)
        
        if len(results) == 0:
            output_results.append([v['reference'], ''.join(v['tokens']), '', ' line_above'])
            exception_results.append([v['reference'], ''.join(v['tokens']), '', ' line_above'])
            exception_counts.append([v['reference'], 0])
        else:
            
            if len(results) != 1:
                exception_counts.append([v['reference'], len(results)])

            for rn, r in enumerate(results):
                if rn == 0:
                    output_results.append([v['reference'], ''.join(v['tokens']), r[1], ' line_above'])
                    if len(results) > 1:
                        exception_results.append([v['reference'], ''.join(v['tokens']), r[1], ' line_above'])
                else:
                    output_results.append(['', '', r[1], ''])
                    if len(results) > 1:
                        exception_results.append(['', '', r[1], ''])
                    
    
    f = open(OUTPUT_FOLDER + output_label + '.matching_bible.html', 'w', encoding='utf-8')
    f.write(t.render(output_label=output_label, output_results=output_results))
    f.close()
    
    f = open(OUTPUT_FOLDER + output_label + '.EXCEPTIONS.matching_bible.html', 'w', encoding='utf-8')
    f.write(t.render(output_label='EXCEPTIONS.' + output_label, output_results=exception_results))
    f.close()
    
    f = open(OUTPUT_FOLDER + output_label + '.EXCEPTION_COUNTS.matching_bible.html', 'w', encoding='utf-8')
    f.write(t_counts.render(output_label='EXCEPTION_COUNTS.' + output_label, output_results=exception_counts))
    f.close()
    
    print(output_label, 'done!')
    
# -------------------------------------------------
    
make_matching_bible(all_verses['KJV'], tcp_kjv, 'KJV_KJV')
make_matching_bible(all_verses['Geneva'], tcp_geneva, 'Geneva_Geneva')

make_matching_bible(all_verses['Geneva'], tcp_kjv, 'Geneva_KJV')
make_matching_bible(all_verses['KJV'], tcp_geneva, 'KJV_Geneva')

KJV_KJV done!
Geneva_Geneva done!
Geneva_KJV done!
KJV_Geneva done!
